# Battleship Architecture

1. 2명의 플레이어가 존재한다.
2. 가정은 서로 HTTP 통신으로 하지만, 통신의 구현은 하지 않는다. 
3. 모든 배는 서로 붙어 있을 수 있다. 
4. 배는 서로 각기 다른 형태를 갖고 있다.

In [243]:
from pprint import pprint
import random

class Player:
    def __init__(self, size=(10, 10)):
        '''
        @param size: (y, x)
        '''
        self.height, self.width = size
        self.matrix = [[0]*size[1] for _ in range(size[0])]
        self.ships = dict()
        self.count = 0
    
    def add_ship(self, width, height):
        while True:
            min_x = random.randint(0, self.width - width)
            min_y = random.randint(0, self.height - height)
            max_x = min_x + width
            max_y = min_y + height
#             print('SHIP:', min_x, min_y, max_x, max_y)
            if self.validate(min_x, min_y, max_x, max_y):
                self._add_ship(min_x, min_y, max_x, max_y)
                break
                
    def _add_ship(self, min_x, min_y, max_x, max_y):
#         print('_add_ship', min_x, min_y, max_x, max_y)
        ship_id = len(self.ships) + 1
        self.ships[ship_id] = (max_x - min_x) * (max_y - min_y)
        self.count += self.ships[ship_id]
        
        for i in range(min_y, max_y):
            for j in range(min_x, max_x):
#                 print('WRITE', i, j)
                self.matrix[i][j] = ship_id
        
                
    def validate(self, min_x, min_y, max_x, max_y):
        if min_x < 0 or min_y < 0:
            return False
        if max_x > self.width or max_y > self.height:
            return False
        
        for i in range(min_y, max_y):
            for j in range(min_x, max_x):
#                 print('validate:', i, j, self.matrix[i][j])
                if self.matrix[i][j] != 0:
#                     print('WRONG', i, j)
                    return False
        return True
    
    def bomb(self, x, y):
        '''
        @return 'INVALID': invalid place to bomb
                'SEA': There is nothing but water
                'BOMB': ship has been bombed
                'SINK': ship has beed sinked
                'END' : Game Over
                
        '''
        value = self.matrix[y][x]
        self.matrix[y][x] = -1
        if value > 0:
            self.ships[value] -= 1
            self.count -= 1
            
        if self.count <= 0:
            return 'END'
        elif value > 0 and self.ships[value] <= 0:
            return 'SINK {0}'.format(value)
        elif value == 0:
            return 'SEA'
        elif value < 0:
            return 'INVALID'
        elif value > 0:
            return 'BOMB x:{0} y:{1} ship:{2}'.format(x, y, value)
    
    def random_bomb(self):
        options = []
        for i in range(self.height):
            for j in range(self.width):
                if self.matrix[i][j] != -1:
                    options.append((i, j))
                    
        y, x = random.choice(options)
        return self.bomb(x, y)
                
        
player1 = Player(size=(5, 5))
player1.add_ship(3, 1)
player1.add_ship(1, 4)
player1.add_ship(1, 1)
player1.add_ship(2, 2)
pprint(player1.matrix)
print(player1.ships)
print(player1.count)


[[0, 2, 0, 4, 4],
 [0, 2, 0, 4, 4],
 [0, 2, 1, 1, 1],
 [0, 2, 3, 0, 0],
 [0, 0, 0, 0, 0]]
{1: 3, 2: 4, 3: 1, 4: 4}
12


In [268]:
print(player1.random_bomb())
pprint(player1.matrix)

END
[[-1, -1, -1, -1, -1],
 [-1, -1, -1, -1, -1],
 [-1, -1, -1, -1, -1],
 [-1, -1, -1, -1, -1],
 [-1, -1, -1, -1, -1]]
